In [ ]:
######BL Snyder 10/11/19
# Fire_MODIS_Thermal_Anomalies
# MODIS Thermal Anomalies/Fire products are primarily derived from MODIS 4- and 11-micrometer radiances.
# The fire detection strategy is based on absolute detection of a fire (when the fire strength is sufficient to detect),
# and on detection relative to its background (to account for variability of the surface temperature and reflection by sunlight).

import requests,json,datetime, time # need to install these libraries to run
from arcgis.gis import GIS
from arcgis import features
from arcgis.features import FeatureLayerCollection

##All TWC visual products have a product name and number
prod_num=str(603) # Radar Derived 24-Hour HailVision. 
APIKEY='THE_WEATHER_COMPANY_DATA_PACKAGE_KEY_HERE'  #This is IBM internal  key 


########################PRODUCT INVENTORY CALL################################  
def InvCall():
    base1 = 'https://api.weather.com/v2/vector-api/products/'+prod_num+'/info?meta=true&max-times=12'
    base2 = '&apiKey='+APIKEY
    url = base1 + base2
    print(url)
    APIcall = requests.get(url) #Our API Call     to grab the latest forecast
    if APIcall.status_code == 200:  #if(APIcall.ok):
        data = APIcall.json()  # Just get most current run and next forecast
        t = data["products"][prod_num]["time"][0]        
        print (t)
        return t
                    
##########################   API CALL   ###################################
def APIcall(t, ts): # function for pulling data 

    fc = {
        "type": "FeatureCollection",
        "features": []
    }
             
    base1 = 'https://api.weather.com/v2/vector-api/products/' + prod_num + '/features?time='+t 
    base2 = '&lod=0&x=0&y=0&tile-size=256&apiKey='+ APIKEY
    url = base1 + base2# make API URL   
    print (url)  
    data = requests.get(url).json() #Our API Call
    for y in data['features']:           
        f = {"type": "Feature", "properties": {}, "geometry": None}
        f['geometry'] = y['geometry']
        f['properties']['validTime'] = y['properties']['validTime']
        f['properties']['confidence'] = y['properties']['confidence']
        f['properties']['date'] = y['properties']['date']
        f['properties']['brightness'] = y['properties']['brightness']
        f['properties']['time'] = y['properties']['time']
        fc['features'].append(f)           
        
    with open("FireThermalAnomalies.geojson", "w") as outfile:
        json.dump(fc, outfile)
    ################################################################################
    ##  ADD CONTENT TO ARCGIS  
    # Connect to the GIS Using Your Credentials
    portalURL = "http://ibmmain.maps.arcgis.com"
    userID = "ESRI_USERNAME"
    password = "ESRI_PASSWORD"
    gis = GIS(portalURL, userID, password)
    #Update each piece of content
    existing_ID = '7dc6e8fa5d2f482b91550a90c2ba3f36'  # Map ID
    wx_item = gis.content.get(existing_ID)
    collection = features.FeatureLayerCollection.fromitem(wx_item)
    collection.manager.overwrite('FireThermalAnomalies.geojson')  
    print ("Updating Layer")


################################################################################
t = InvCall()
print(t)
fmt = "%Y-%m-%d %H:%M:%S"
t_utc = datetime.datetime.utcfromtimestamp(int(t)/1000)
t_utc_formatted = t_utc.strftime(fmt)
APIcall(t, t_utc_formatted)